<a href="https://colab.research.google.com/github/rezzycavalheiro/FERMachineLearning/blob/master/TCC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is a prototype for my final course project. 

**Course**: Computer Science  
**University**: Pontifical Catholic University of Paraná (PUC PR)  
**Student's name**: Renata Cavalheiro da Silva  
    
**Project's title**: Classification of Facial Expressions Using Deep Learning

> **Description**: This project aims to build a Facial Expresion Recognition (FER) algorithm that presents a high accuracy and that could be adapted for psychologists and psychiatrists to use during their appointments with their pacients. The idea is that this tool would help the medical professionals to record and evaluate the expressions presented by their pacients, and use the data to check if they match what the pacient is saying. This is due to people being able to say something when they are feeling another - like someone with depressive symptoms saying they don't feel sadness, but in their expression actually showing this emotion. 






In [0]:
# ACCESSING THE FOLDER ON MY DRIVE AND GETTING ITS CONTENT

# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("Auth Success")

# Getting the id of the folder which contains the files 
folder_id = '1vtV4IahA5vu3P8WQRRTPgRK8qPr1umKH'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()

for item in lister:
    print(item['title']) 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {
        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[file['mimeType']]
        item.GetContentFile(file['title'], mimetype=download_mimetype)

        item.GetContentFile(r'C:\Users\Renata\Desktop\googedrive\\' + item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(r'C:\Users\Renata\Desktop\googedrive\\' + item['title'])

Auth Success
fer2013.csv
title: fer2013.csv, mimeType: text/csv
README
title: README, mimeType: application/octet-stream
fer2013.bib
title: fer2013.bib, mimeType: text/x-bibtex


In [0]:
# READING THE FILE

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file

fer2013 = pd.read_csv(r'C:\Users\Renata\Desktop\googedrive\\fer2013.csv')
print(fer2013)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [0]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt